# Evalluate fine tune v2 model with Exact Match Accuracy

In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import torch


In [ ]:

model_name = "khalifa1/Sql_LLama2_V3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [8]:

# GDrive Location for Train/Test Data
# GDrive Location for Train/Test Data
DATA_PATH ="/content/drive/MyDrive/Text2SQL/Text2SQL/Data/"
DS_DIR = "sql_train_test"
PKL_DIR = "test/"
PKL_FILE ="sql_test.pkl"
LLMRES_PKL_FILE="sql_test_finev2.pkl"

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test_df = pd.read_pickle(DATA_PATH + PKL_DIR + PKL_FILE)


In [ ]:
!pip install -q -U datasets


In [ ]:
from datasets import load_dataset, load_from_disk


In [ ]:

# Load Test & Training Data from Disk
dataset = load_from_disk(DATA_PATH + DS_DIR)



In [ ]:
display(dataset['train'])
display(dataset['test'])

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 4086
})

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 454
})

In [ ]:

display(test_df.head(3))
display(test_df.shape)


,response,question,context,__index_level_0__,text
0,SELECT College FROM match_season GROUP BY Coll...,Show the name of colleges that have at least t...,CREATE TABLE match_season (College VARCHAR),611,### Instruction:\n You are a powerful text-...
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",2556,### Instruction:\n You are a powerful text-...
2,"SELECT AVG(T1.price), T2.name FROM products AS...",Find the average prices of all products from e...,"CREATE TABLE products (price INTEGER, Manufact...",3026,### Instruction:\n You are a powerful text-...


(454, 5)

In [ ]:
print(model.get_memory_footprint()/1e9) # GB


13.477093632


In [ ]:
! pip install -q -U langchain
!pip install -q -U langchain-community  # Install langchain-community



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [ ]:
from langchain_community.llms import HuggingFacePipeline # Import from the correct location


In [ ]:

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=180,
    do_sample = True,
    temperature=0.1,
    top_k=4,
    # top_p=0.95,
    repetition_penalty=1.2
)


llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
<ipython-input-30-04751c614709>:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
ID = 10
test_df['text'][ID]


'### Instruction:\n    You are a powerful text-to-SQL model.     Your job is to answer questions about a database.     You are given a question and context regarding one or more tables.\n\n    You must output the SQL query that answers the question.\n\n    ### Input:\n    What is the average amount due for all the payments?\n    ### Context:\n    CREATE TABLE payments (amount_due INTEGER)\n    ### Response:\n    '

In [ ]:
display(test_df['response'][ID])


'SELECT AVG(amount_due) FROM payments'

In [ ]:

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


In [ ]:

def parse(text):
    start_marker = '### Response:'
    end_marker = '### End'
    start_index = text.find(start_marker)
    end_index = text.find(end_marker, start_index + len(start_marker))

    return (text[start_index + len(start_marker):].strip() if start_index != -1 and end_index == -1
            else text[start_index + len(start_marker):end_index].strip() if start_index != -1
            else None)

In [ ]:

def gen_eval_res(test_df):
  eval_res = []
  time_res = []
  row_no = 1
  # tmp = test_df.iloc[:2]
  for row in test_df.itertuples(index=False):
    start_time = perf_counter()
    # print(row.text)
    text = row.text # test_df['text'][ID]
    llm_res = pipe(text)[0]['generated_text']
    parse_res = parse(llm_res)
    end_time = perf_counter()
    eval_res.append(parse_res)
    time_res.append(end_time-start_time)
    if row_no % 10 == 0:
      print(f"*** Row {row_no} Done ***")
    row_no += 1

  test_df['eval_res'] = eval_res
  test_df['eval_time'] = time_res

In [ ]:

text = test_df['text'][ID]
llm_res = pipe(text)[0]['generated_text']
parse_res = parse(llm_res)
# print(llm_res)
print(parse_res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SELECT AVG(p.amount_due) AS average_amount_due FROM payments p;


In [ ]:
from time import perf_counter


In [ ]:

eval_start_time = perf_counter()
gen_eval_res(test_df)
eval_end_time = perf_counter()
print('Evaluation Took: ', round(eval_end_time-eval_start_time, 2))

In [ ]:
test_df.to_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE)


In [9]:
import pandas as pd
test_df2=pd.read_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE)
display(test_df2.head(10))
test_df2.shape

,response,question,context,__index_level_0__,text,eval_res,eval_time
0,SELECT College FROM match_season GROUP BY Coll...,Show the name of colleges that have at least t...,CREATE TABLE match_season (College VARCHAR),611,### Instruction:\n You are a powerful text-...,SELECT DISTINCT m.college FROM match_season m ...,2.292940
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",2556,### Instruction:\n You are a powerful text-...,SELECT p.name FROM Person p JOIN PersonFriend ...,3.423700
2,"SELECT AVG(T1.price), T2.name FROM products AS...",Find the average prices of all products from e...,"CREATE TABLE products (price INTEGER, Manufact...",3026,### Instruction:\n You are a powerful text-...,"SELECT p.manufacturer AS company_name, AVG(p.p...",4.605104
3,"SELECT Support_rate, Consider_rate, Oppose_rat...","Please list support, consider, and oppose rate...","CREATE TABLE candidate (Support_rate VARCHAR, ...",1438,### Instruction:\n You are a powerful text-...,"SELECT c.name, CAST(c.support_rate AS FLOAT) A...",6.660659
4,"SELECT Fname, Lname FROM Student WHERE Age > 1...",List all female students age is older than 18 ...,"CREATE TABLE Student (Fname VARCHAR, Lname VAR...",1676,### Instruction:\n You are a powerful text-...,"SELECT s.fname, s.lname FROM student s WHERE C...",2.951694
5,"SELECT payment_method_code, COUNT(*) FROM cust...","For each payment method, return how many custo...",CREATE TABLE customers (payment_method_code VA...,2726,### Instruction:\n You are a powerful text-...,"SELECT c.payment_method_code, COUNT(c.customer...",3.106380
6,SELECT AVG(age) FROM student WHERE NOT stuid I...,Find the average age of students who do not ha...,"CREATE TABLE student (age INTEGER, stuid VARCH...",4007,### Instruction:\n You are a powerful text-...,SELECT AVG(s.age) FROM student s LEFT JOIN has...,2.178076
7,"SELECT Team, COUNT(*) FROM elimination GROUP B...",Show different teams in eliminations and the n...,CREATE TABLE elimination (Team VARCHAR),1081,### Instruction:\n You are a powerful text-...,"SELECT e1.team, COUNT(e2.team) AS num_eliminat...",3.616365
8,"SELECT T2.name, COUNT(*) FROM exhibition AS T1...",Show all artist names and the number of exhibi...,"CREATE TABLE artist (name VARCHAR, artist_id V...",952,### Instruction:\n You are a powerful text-...,"SELECT a.""name"", COUNT(e.artist_id) AS num_exh...",3.659992
9,SELECT job_id FROM job_history WHERE end_date ...,display job ID for those jobs that were done b...,"CREATE TABLE job_history (job_id VARCHAR, end_...",2050,### Instruction:\n You are a powerful text-...,SELECT jh.job_id FROM job_history jh WHERE CAS...,3.782999


(454, 7)

In [18]:
import re
import pandas as pd
from difflib import SequenceMatcher
def normalize_text(text):
    """Remove punctuation, convert to lowercase, and strip extra spaces."""
    return re.sub(r'\W+', ' ', text).strip().lower()

def calculate_exact_match(df, threshold=0.90):
    def is_match(response, eval_res):
        normalized_response = normalize_text(response)
        normalized_eval = normalize_text(eval_res)
        similarity = SequenceMatcher(None, normalized_response, normalized_eval).ratio()
        return similarity >= threshold

    df['exact_match'] = df.apply(lambda row: is_match(row['response'], row['eval_res']), axis=1)
    accuracy = df['exact_match'].sum() / len(df)
    mismatches = df[~df['exact_match']]
    print(f"Exact Match Accuracy: {accuracy:.2%}")
    print(f"\nSample Mismatches ({len(mismatches)} cases):")

    return mismatches.sample(min(3, len(mismatches)))


In [19]:

mismatches = calculate_exact_match(test_df2)
display(mismatches)

Exact Match Accuracy: 70.48%

Sample Mismatches (134 cases):


,response,question,context,__index_level_0__,text,eval_res,eval_time,exact_match
400,"SELECT T1.name_first, T1.name_last, T1.player_...",Which 3 players won the most player awards? Li...,"CREATE TABLE player (name_first VARCHAR, name_...",2142,### Instruction:\n You are a powerful text-...,SELECT p.name_first ||'' || p.name_last AS ful...,5.433262,False
336,SELECT AVG(LifeExpectancy) FROM country WHERE ...,What is the average life expectancy in African...,"CREATE TABLE country (LifeExpectancy INTEGER, ...",4357,### Instruction:\n You are a powerful text-...,71,0.236284,False
30,SELECT Gname FROM Plays_games AS T1 JOIN Video...,Show all game names played by Linda Smith,"CREATE TABLE Student (Stuid VARCHAR, Lname VAR...",3444,### Instruction:\n You are a powerful text-...,SELECT DISTINCT g.title FROM Games g JOIN Play...,2.848596,False
